<h1> <b> Purpose of this analysis</b></h1>
Purpose of this mini project is to experiment some of data wrangling techniques. We have rcieved Lending Club Loan Data set from 2007 to 2015. There are total 2.2 million rows and 145 columns. It is important to do some pre-processing work in order to analyze data and fill out missing values.  

<h2> <b> Packages</b></h2>
We will start by importing some of packages. Following packages will be imported.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

sns.set_style('whitegrid')

<h2> Step 1 : Reading Data in Dataframe </h2>
Now we will be reading data from "Loan.csv" to Pandas Dataframe.

In [3]:
file1 = "loan.csv"
ChunkSize = 200000
i = 1
for chunk in pd.read_csv(file1, chunksize=ChunkSize, low_memory=False):
    loan_data = chunk if i == 1 else pd.concat([loan_data, chunk])
    print('-->Read Chunk...', i)
    i += 1


# data = pd.read_csv("loan.csv", low_memory=False)



-->Read Chunk... 1
-->Read Chunk... 2
-->Read Chunk... 3
-->Read Chunk... 4
-->Read Chunk... 5
-->Read Chunk... 6
-->Read Chunk... 7
-->Read Chunk... 8
-->Read Chunk... 9
-->Read Chunk... 10
-->Read Chunk... 11
-->Read Chunk... 12


<h2> Step 2 : Checking data size </h2>
Let's quickly check the shape of data.This will give us idea as how large is current dataset. As we can see, current data set contains more than 2 million rows and 145 columns. Not all columns are useful for our analysis. 

In [4]:
# print(list(data.columns))
print(loan_data.shape)

(2260668, 145)


<h2> Step 3 : Calculating Percentage of missing data per column </h2>
Here  we will be creating Dataframe called "df_null". This Data frame consist of column name with percent of missing data in each column in descending order. It is noted that any column that has more than 60% of data missing are useless for our analysis and we will simply drop it. 
As we see some column like "url", "id", "member_id" has 100% missing values. Apart from that lot of columns have more than 60% missing data. We will simply drop column which has more than 60% data missing. 

In [5]:
# print((data.isna().sum()[data.isna().sum() > 0]))
df_null = pd.DataFrame({'Count': loan_data.isnull().sum(), 'Percent': 100*loan_data.isnull().sum()/len(loan_data)})
print(df_null[df_null['Percent'] >= 60].sort_values(by='Percent', ascending=False).round(1))



# print(data.head(3))

                                              Count  Percent
id                                          2260668    100.0
url                                         2260668    100.0
member_id                                   2260668    100.0
orig_projected_additional_accrued_interest  2252242     99.6
hardship_dpd                                2250055     99.5
hardship_length                             2250055     99.5
hardship_reason                             2250055     99.5
hardship_status                             2250055     99.5
deferral_term                               2250055     99.5
hardship_amount                             2250055     99.5
hardship_start_date                         2250055     99.5
hardship_end_date                           2250055     99.5
payment_plan_start_date                     2250055     99.5
hardship_loan_status                        2250055     99.5
hardship_type                               2250055     99.5
hardship_payoff_balance_

<h2> Step 4 : Dropping column with more than 60% missing data </h2>
We will create a list of columns that a data missing more than 60%. We will simply drop those columns and create new dataframe called df_clean. We wil print shape of new dataframe to see how much data has been reduced. We can see total count of columns from 145 to reduced to 103. 

In [6]:
# Creating list of column with more than 60% data missing. 
list_60 = list(df_null[df_null['Percent']>60].index)

print(len(list_60))
df_clean = loan_data.drop(list_60, axis=1)
print(df_clean.shape)

42
(2260668, 103)


<h2> Step 5 : Analyzing columns with missing data  more than 10% and 60% </h2>
Now that we have eliminated major columns that had missing values. Lets analyze remaining column with missing values. For that we will print all column that has missing data and see how we can predict missing values. We will predict all columns that has missing data grether than 10%. Since we already dropped columns that has missing data more than 60%, in our dataframe.

In [7]:
df_null = pd.DataFrame({'Count': df_clean.isnull().sum(), 'Percent': 100*df_clean.isnull().sum()/len(df_clean)})
print(df_null[df_null['Percent'] > 10])


                          Count    Percent
mths_since_last_delinq  1158502  51.246003
next_pymnt_d            1303607  57.664681
open_acc_6m              866130  38.313012
open_act_il              866129  38.312968
open_il_12m              866129  38.312968
open_il_24m              866129  38.312968
mths_since_rcnt_il       909924  40.250227
total_bal_il             866129  38.312968
il_util                 1068850  47.280273
open_rv_12m              866129  38.312968
open_rv_24m              866129  38.312968
max_bal_bc               866129  38.312968
all_util                 866348  38.322655
inq_fi                   866129  38.312968
total_cu_tl              866130  38.313012
inq_last_12m             866130  38.313012
mths_since_recent_inq    295435  13.068482


<h2> Step 5A : Checking data in columns with missing data more than 10% </h2>
Lets examine few values of this columns so we can get idea what kind of data estimation we need to do.

In [8]:
col_10 = list(df_null[df_null['Percent'] > 10].index)
print(df_clean[col_10].head(3))

   mths_since_last_delinq next_pymnt_d  open_acc_6m  open_act_il  open_il_12m  \
0                     NaN     Mar-2019          2.0          2.0          1.0   
1                    71.0     Mar-2019          4.0          4.0          2.0   
2                     NaN     Mar-2019          0.0          1.0          0.0   

   open_il_24m  mths_since_rcnt_il  total_bal_il  il_util  open_rv_12m  \
0          2.0                 2.0       12560.0     69.0          2.0   
1          3.0                 3.0       87153.0     88.0          4.0   
2          2.0                14.0        7150.0     72.0          0.0   

   open_rv_24m  max_bal_bc  all_util  inq_fi  total_cu_tl  inq_last_12m  \
0          7.0      2137.0      28.0     1.0         11.0           2.0   
1          5.0       998.0      57.0     2.0         15.0           2.0   
2          2.0         0.0      35.0     1.0          5.0           0.0   

   mths_since_recent_inq  
0                    2.0  
1                    4.

<h2> Step 5B : Dropping payment dates columns </h2>
As we see above data set most missing data is numeric and we can use mode or median in order to fill missing values. By looking at dataset and name of columns, mode will be better option compared to median for most of columns. For "total_bil_il" we will use median. Lets analyze mode and median of those columns. But before we need to drop 3 columns which is not useful for our analysis. Those are 
1)  "next_pymnt_dt
2) 'last_credit_pull_d'
3) 'last_pymnt_d'

Lets create a list of these columns. 

In [9]:
# print(df_clean.shape)
cols_to_drop = ['next_pymnt_d', 'last_credit_pull_d', 'last_pymnt_d']
df_clean.drop(cols_to_drop, axis=1, inplace=True)
df_null = pd.DataFrame({'Count': df_clean.isnull().sum(), 'Percent': 100*df_clean.isnull().sum()/len(df_clean)})

<h2> Step 5C : Checking mode and median of missing data columns </h2>
Now we will eamine mode and median of all columns those we trying to do estiamtion.



In [10]:
col_10 = list(df_null[df_null['Percent'] > 10].index)
# print(col_10)
# print(df_null.loc[col_10, :])
for index in col_10:
    df_null.loc[index, 'mode'] = df_clean[index].mode()[0]
    df_null.loc[index, 'median'] = df_clean[index].median()
    df_null.loc[index, 'mean'] = df_clean[index].mean()
print(df_null.loc[col_10, :].round(1))



                          Count  Percent  mode   median     mean
mths_since_last_delinq  1158502     51.2  12.0     31.0     34.5
open_acc_6m              866130     38.3   0.0      1.0      0.9
open_act_il              866129     38.3   1.0      2.0      2.8
open_il_12m              866129     38.3   0.0      0.0      0.7
open_il_24m              866129     38.3   1.0      1.0      1.6
mths_since_rcnt_il       909924     40.3   7.0     13.0     21.2
total_bal_il             866129     38.3   0.0  23127.0  35506.6
il_util                 1068850     47.3  78.0     72.0     69.1
open_rv_12m              866129     38.3   0.0      1.0      1.3
open_rv_24m              866129     38.3   1.0      2.0      2.7
max_bal_bc               866129     38.3   0.0   4413.0   5806.4
all_util                 866348     38.3  59.0     58.0     57.0
inq_fi                   866129     38.3   0.0      1.0      1.0
total_cu_tl              866130     38.3   0.0      0.0      1.5
inq_last_12m             

<h2> Step 5D : predicting missing values using mode </h2>
By looking at above table, it makes more sense to fill all above columns by mode rather than median. Lets fill those missing values in above columns using mode.

In [11]:
df_clean[col_10] = df_clean[col_10].fillna(df_clean.mode().iloc[0])

<h2> Step 6 : Dropping all missing data </h2>
Now lets examine remaining missing data. Most columns will have missing data less than 10%. We will simply drop them. We may see lot of columns having missing data less than 10%. We will simply drop them. That way we will only lose 10% of data

In [12]:
df_clean.dropna(inplace=True)
print(df_clean.shape)


(1854073, 100)


# Get year info from Issue Date

In [13]:
df_clean['year'] = pd.to_datetime(df_clean['issue_d']).dt.year
df_clean = df_clean.drop('issue_d', axis=1)
df_clean['year'].head()

0    2018
1    2018
2    2018
3    2018
4    2018
Name: year, dtype: int64

In [14]:
df_clean['percent_complete'] = (df_clean['total_pymnt']*100 / df_clean['loan_amnt']).round(2)
# print(df_clean[df_clean['loan_status'] == "Fully Paid"]['percent_complete'].head())

In [15]:

df_clean['open_acc_6m'] = df_clean['open_acc_6m'].apply(lambda x: 3 if x>=3 else x)
df_clean['open_act_il'] = df_clean['open_act_il'].apply(lambda x: 5 if x>=5 else x)
df_clean['open_il_12m'] = df_clean['open_il_12m'].apply(lambda x: 3 if x>=3 else x)
df_clean['open_rv_12m'] = df_clean['open_rv_12m'].apply(lambda x: 3 if x>=3 else x)
df_clean['inq_fi'] = df_clean['inq_fi'].apply(lambda x: 3 if x>=3 else x)
df_clean['inq_last_12m'] = df_clean['inq_last_12m'].apply(lambda x: 4 if x>=4 else x)
df_clean['acc_open_past_24mths'] = df_clean['acc_open_past_24mths'].apply(lambda x: 8 if x>=8 else x)
df_clean['mort_acc'] = df_clean['mort_acc'].apply(lambda x: 4 if x>=4 else x)
df_clean['num_accts_ever_120_pd'] = df_clean['num_accts_ever_120_pd'].apply(lambda x: 3 if x>=3 else x)
df_clean['num_tl_op_past_12m'] = df_clean['num_tl_op_past_12m'].apply(lambda x: 5 if x>=5 else x)

In [16]:
df_clean['max_bal_bc_cat'] = pd.cut(x=df_clean['max_bal_bc'], bins=[0, 1000, 2000, 5000, 10000 ,1000000],\
                                    include_lowest=True)
df_clean['num_sats_cat'] = pd.cut(x=df_clean['num_sats'], bins=[0, 5, 10, 20, 30 ,150],include_lowest=True)
df_clean['pct_tl_nvr_dlq_cat'] = pd.cut(x=df_clean['pct_tl_nvr_dlq'], bins=[0, 60, 70, 80, 90 ,100],include_lowest=True)
df_clean['percent_bc_gt_75_cat'] = pd.cut(x=df_clean['percent_bc_gt_75'], bins=[0, 20, 40, 60, 80 ,100],include_lowest=True)
df_clean['percent_bc_gt_75_cat'].head()

0    (-0.001, 20.0]
1    (-0.001, 20.0]
2    (-0.001, 20.0]
3     (80.0, 100.0]
4    (-0.001, 20.0]
Name: percent_bc_gt_75_cat, dtype: category
Categories (5, interval[float64]): [(-0.001, 20.0] < (20.0, 40.0] < (40.0, 60.0] < (60.0, 80.0] < (80.0, 100.0]]

# Finding Out redundant values
We will use value_count(normalize=True) function in order to find out redundant values. 

In [17]:
col_list = ['mo_sin_old_il_acct', 'mort_acc', 'num_accts_ever_120_pd', 'num_rev_accts', 'num_rev_tl_bal_gt_0',\
            'num_sats_cat', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', \
           'pct_tl_nvr_dlq_cat', 'percent_bc_gt_75_cat', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', \
           'hardship_flag']
# print(col_list)
print(df_clean['hardship_flag'].value_counts(normalize=True).round(4)*100)
# print(df_clean['num_sats'].nlargest(5))
for c in col_list:
    print ("---- %s ---" % c)
    df_t = df_clean[c].value_counts(normalize=True).round(2)
    
    print("Max value = %f" %df_t.max())
    print("Length = %d" %len(df_t))

N    99.96
Y     0.04
Name: hardship_flag, dtype: float64
---- mo_sin_old_il_acct ---
Max value = 0.010000
Length = 545
---- mort_acc ---
Max value = 0.410000
Length = 5
---- num_accts_ever_120_pd ---
Max value = 0.760000
Length = 4
---- num_rev_accts ---
Max value = 0.060000
Length = 115
---- num_rev_tl_bal_gt_0 ---
Max value = 0.150000
Length = 49
---- num_sats_cat ---
Max value = 0.450000
Length = 5
---- num_tl_120dpd_2m ---
Max value = 1.000000
Length = 7
---- num_tl_30dpd ---
Max value = 1.000000
Length = 5
---- num_tl_90g_dpd_24m ---
Max value = 0.950000
Length = 33
---- num_tl_op_past_12m ---
Max value = 0.260000
Length = 6
---- pct_tl_nvr_dlq_cat ---
Max value = 0.780000
Length = 5
---- percent_bc_gt_75_cat ---
Max value = 0.360000
Length = 5
---- pub_rec_bankruptcies ---
Max value = 0.880000
Length = 12
---- tax_liens ---
Max value = 0.970000
Length = 40
---- tot_hi_cred_lim ---
Max value = 0.000000
Length = 508102
---- hardship_flag ---
Max value = 1.000000
Length = 2


In [18]:
col_list2 = ['pymnt_plan', 'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',\
            'total_acc', 'out_prncp', 'out_prncp_inv',  'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', \
             'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt',\
            'collections_12_mths_ex_med', 'policy_code',  'application_type',  'acc_now_delinq', 'tot_coll_amt', \
            'tot_cur_bal', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'open_rv_24m', 'max_bal_bc', \
             'all_util', 'total_rev_hi_lim', 'total_cu_tl', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', \
             'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op',\
             'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_actv_bc_tl',\
            'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', \
            'num_rev_tl_bal_gt_0', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'pct_tl_nvr_dlq_cat',\
            'pct_tl_nvr_dlq', 'pub_rec_bankruptcies', 'tax_liens', 'total_il_high_credit_limit', 'hardship_flag',\
            'disbursement_method', 'debt_settlement_flag', 'max_bal_bc', 'num_sats', 'pct_tl_nvr_dlq', 'percent_bc_gt_75',\
            'funded_amnt_inv', 'zip_code', 'funded_amnt',  'title', 'earliest_cr_line', ]
print(len(col_list2))
# df_clean['pymnt_plan'].value_counts(normalize=True)



71


<h2> Step 7 : Analyzing datatypes of clean data </h2>
Now we have get rid of all missing data. Dataset is almost clean. Lets see datatypes of datasets. 

In [19]:
# print(df_clean.dtypes)

df_clean2 = df_clean.drop(col_list2, axis=1) 

print(df_clean2.dtypes)

loan_amnt                   int64
term                       object
int_rate                  float64
installment               float64
grade                      object
sub_grade                  object
emp_title                  object
emp_length                 object
home_ownership             object
annual_inc                float64
verification_status        object
loan_status                object
purpose                    object
addr_state                 object
dti                       float64
delinq_2yrs               float64
initial_list_status        object
open_acc_6m               float64
open_act_il               float64
open_il_12m               float64
il_util                   float64
open_rv_12m               float64
inq_fi                    float64
inq_last_12m              float64
acc_open_past_24mths      float64
mort_acc                  float64
num_accts_ever_120_pd     float64
num_tl_op_past_12m        float64
tot_hi_cred_lim           float64
total_bal_ex_m

<h2> Step 8 : Fixing "Employment Length" column </h2>
In above datatypes one of the important title is employment length. We should be expecting it as integer or float but it is object type. Lets analyze this column.

In [20]:
print(df_clean2['emp_length'].value_counts())

10+ years    665309
2 years      179041
3 years      158836
< 1 year     157037
1 year       129053
5 years      121646
4 years      119208
6 years       89893
7 years       81825
8 years       81520
9 years       70705
Name: emp_length, dtype: int64


Now lets remove unnecessary keywords suh as "year", "<" and "+". We will consider all records of empoyment length <1 years as 0 years and 10+ years as 10.

In [21]:
# df_clean['emp_length'].astype('str').dtypes
df_clean2['emp_length'] = df_clean2['emp_length'].str.replace(r'[+]\s|[a-z]', '')
df_clean2['emp_length'] = df_clean2['emp_length'].str.replace('< 1', '0')
# df_clean['emp_length'] = df_clean['emp_length'].str.replace('10+ ', '10')

print(df_clean2['emp_length'].value_counts())



10    665309
2     179041
3     158836
0     157037
1     129053
5     121646
4     119208
6      89893
7      81825
8      81520
9      70705
Name: emp_length, dtype: int64


# Good Loan and Bad Loan

In [22]:
# df_clean2['year'] = pd.to_datetime(df_clean2['issue_d']).dt.year
# print(train_data['loan_status'].value_counts())
# train_data['status_var'] = train_data['loan_status'].map({'Fully Paid':0, 'Charged Off':1, 'Late (31-120 days)':1, \
#                                                         'In Grace Period':1, 'Late (16-30 days)':1, 'Default':1})
# print(train_data['status_var'].value_counts())
loan_status_dict = {"Fully Paid": 0, "Charged Off": 1, "Late (31-120 days)": 1,
                    "In Grace Period": 1, "Late (16-30 days)": 1, "Default": 1, "Current" : 2}
# known_status = df_clean2['loan_status'] != 'Current'
df_clean2["loan_status_count"] = df_clean2["loan_status"].map(loan_status_dict)
df_clean2["loan_status_count"] = df_clean2["loan_status_count"].astype("int")

# loan_known_data = df_clean2[known_status].reset_index()
loan_status_dict = {0.0: "Good loans", 1.0: "Bad loans", 2.0: 'Unknown'}
df_clean2["loan_status"] = df_clean2["loan_status_count"].map(loan_status_dict)

<h2> Step 9 : Analyzing clean data </h2>
Now that we have clean data, we will analyze its shape. Majority of columns are numeric and hence we will use dataframe.describe function to just see summary of numeric columns. 

In [23]:
# df_clean3 = df_clean2.drop('loan_status')
print(df_clean2.shape)
pd.options.display.float_format = '{:.2f}'.format
print(df_clean2.head())

(1854073, 37)
   loan_amnt        term  int_rate  installment grade sub_grade  \
0       2500   36 months     13.56        84.92     C        C1   
1      30000   60 months     18.94       777.23     D        D2   
2       5000   36 months     17.97       180.69     D        D1   
3       4000   36 months     18.94       146.51     D        D2   
4      30000   60 months     16.14       731.78     C        C4   

        emp_title emp_length home_ownership  annual_inc verification_status  \
0            Chef         10           RENT    55000.00        Not Verified   
1     Postmaster          10       MORTGAGE    90000.00     Source Verified   
2  Administrative         6        MORTGAGE    59280.00     Source Verified   
3   IT Supervisor         10       MORTGAGE    92000.00     Source Verified   
4        Mechanic         10       MORTGAGE    57250.00        Not Verified   

  loan_status             purpose addr_state   dti  delinq_2yrs  \
0     Unknown  debt_consolidation        

<h2> Step 10: Exporting clean data to 'csv"

In [24]:
df_clean2.to_csv("Loan_cleandata2.csv", index=False)